In [1]:
import random
import torch
import torchmetrics

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import transforms


In [2]:
# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Example for RGB images
# ])

In [3]:
x = np.load("./data/balanced/input_data.npy")
labels = np.load("./data/balanced/label_data.npy", allow_pickle=True)

In [4]:
x_tensor = torch.from_numpy(x)
x_t = x_tensor.view(-1, 3, 49, 49)
inputs = x_t.float()


In [5]:
unique_labels = np.unique(labels)
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
print("Label Mapping:", label_mapping)
integer_labels = np.array([label_mapping[label] for label in labels])
targets = torch.from_numpy(integer_labels)
targets = targets.long()

Label Mapping: {'Lg': 0, 'P': 1, 'Pg': 2, 'Pn': 3, 'S': 4, 'Sn': 5}


In [6]:
# x_min = inputs.min()
# x_max = inputs.max()
# print((x_min, x_max))

# inputs = 2 * (inputs - x_min) / (x_max - x_min) -1

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [8]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(
            in_channels=32, out_channels=64, kernel_size=3, padding=1
        )
        self.conv3 = nn.Conv2d(
            in_channels=64, out_channels=128, kernel_size=3, padding=1
        )

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        # self.dropout = nn.Dropout(p=0.25)

        self.fc1 = nn.Linear(128 * 6 * 6, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 6)

    def forward(self, x):
        conv1 = self.pool(F.leaky_relu(self.conv1(x)))  # torch.Size([13042, 16, 24, 24])
        conv2 = self.pool(F.tanh(self.conv2(conv1)))  # torch.Size([13042, 32, 12, 12])
        conv3 = self.pool(F.tanh(self.conv3(conv2)))  # torch.Size([13042, 64, 6, 6])
        
        # dropout = self.dropout(conv3)

        flatten = conv3.view(-1, 128 * 6 * 6)

        fc1 = F.tanh(self.fc1(flatten))
        fc2 = F.tanh(self.fc2(fc1))
        fc3 = F.tanh(self.fc3(fc2))

        return fc3

In [9]:
model = CNN()
model = model.to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Rprop(model.parameters(), lr=0.0006)

warmup_steps = 5
epochs = 50

schedular = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

inputs = inputs.to(device=device)
targets = targets.to(device=device)

for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model.forward(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()
    
    print(f"Epoch {epoch+1}/{epochs}, Cross Entropy Loss: {loss.item()}")

Epoch 1/50, Cross Entropy Loss: 1.795331358909607
Epoch 2/50, Cross Entropy Loss: 1.7775285243988037
Epoch 3/50, Cross Entropy Loss: 1.7513447999954224
Epoch 4/50, Cross Entropy Loss: 1.7278757095336914
Epoch 5/50, Cross Entropy Loss: 1.6988202333450317
Epoch 6/50, Cross Entropy Loss: 1.686674952507019
Epoch 7/50, Cross Entropy Loss: 1.6643508672714233
Epoch 8/50, Cross Entropy Loss: 1.6566336154937744
Epoch 9/50, Cross Entropy Loss: 1.6358985900878906
Epoch 10/50, Cross Entropy Loss: 1.6240592002868652
Epoch 11/50, Cross Entropy Loss: 1.6083415746688843
Epoch 12/50, Cross Entropy Loss: 1.593199610710144
Epoch 13/50, Cross Entropy Loss: 1.5827605724334717
Epoch 14/50, Cross Entropy Loss: 1.5719510316848755
Epoch 15/50, Cross Entropy Loss: 1.5601954460144043
Epoch 16/50, Cross Entropy Loss: 1.5493184328079224
Epoch 17/50, Cross Entropy Loss: 1.5385733842849731
Epoch 18/50, Cross Entropy Loss: 1.527693271636963
Epoch 19/50, Cross Entropy Loss: 1.515462040901184
Epoch 20/50, Cross Entropy

In [10]:
x = np.load("./data/balanced/test_data.npy")
labels = np.load("./data/balanced/test_label_data.npy", allow_pickle=True)

x_tensor = torch.from_numpy(x)
x_t = x_tensor.view(-1, 3, 49, 49)
inputs = x_t.float()
inputs = inputs.to(device)

unique_labels = np.unique(labels)
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
print("Label Mapping:", label_mapping)
integer_labels = np.array([label_mapping[label] for label in labels])
targets = torch.from_numpy(integer_labels)
targets = targets.long()
targets = targets.to(device=device)

Label Mapping: {'Lg': 0, 'P': 1, 'Pg': 2, 'Pn': 3, 'S': 4, 'Sn': 5}


In [11]:
y_pred = model(inputs)

In [12]:
probabilities = F.softmax(y_pred, dim=1)

# Get the predicted class with the highest probability
predicted_class = torch.argmax(probabilities, dim=1)

In [13]:
predicted_class = predicted_class.to(device=device)
print(targets)

tensor([4, 1, 0,  ..., 5, 0, 5], device='cuda:0')


In [14]:
accuracy_metric  = torchmetrics.Accuracy(task="multiclass", num_classes=6).to(device=device)
accuracy = accuracy_metric(predicted_class, targets)

In [15]:
accuracy

tensor(0.4145, device='cuda:0')